In [17]:
import pandas as pd
from paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location,pickup_longitude,pickup_latitude,target_rides_next_hour
0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,3.0,1.0,...,0.0,3.0,1.0,1.0,0.0,2024-01-29,Albany Park,-87.721559,41.968069,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,1.0,3.0,1.0,2.0,1.0,2024-01-30,Albany Park,-87.721559,41.968069,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,...,3.0,0.0,2.0,0.0,0.0,2024-01-31,Albany Park,-87.721559,41.968069,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,1.0,...,0.0,2.0,1.0,0.0,0.0,2024-02-01,Albany Park,-87.721559,41.968069,0.0
4,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,0.0,...,0.0,2.0,1.0,4.0,1.0,2024-02-02,Albany Park,-87.721559,41.968069,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-27,Andersonville,-87.671446,41.979796,0.0
35051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-28,Andersonville,-87.671446,41.979796,0.0
35052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-29,Andersonville,-87.671446,41.979796,0.0
35053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2025-01-30,Andersonville,-87.671446,41.979796,0.0


In [18]:
from datetime import datetime
from data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2024, 11, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(26315, 676)
y_train.shape=(26315,)
X_test.shape=(8740, 676)
y_test.shape=(8740,)


In [19]:
import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_1_hour']

In [20]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
predictions

0       0.0
1       0.0
2       1.0
3       1.0
4       1.0
       ... 
8735    0.0
8736    0.0
8737    0.0
8738    0.0
8739    0.0
Name: rides_previous_1_hour, Length: 8740, dtype: float32

In [21]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=1.6330


In [22]:
import pandas as pd
import numpy as np

class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t - 7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_{7*24}_hour']

In [23]:

model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [24]:
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=1.9455


In [25]:
class BaselineModelLast4Weeks:
    """
    Prediction = actual demand observed at t - 7 days, t - 14 days, t - 21 days, t - 28 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> pd.Series:
        """"""
        return 0.25*(
            X_test[f'rides_previous_{7*24}_hour'] + \
            X_test[f'rides_previous_{2*7*24}_hour'] + \
            X_test[f'rides_previous_{3*7*24}_hour'] + \
            X_test[f'rides_previous_{4*7*24}_hour']
        )

In [26]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=1.7751
